In [1]:
import numpy as np
import netCDF4 as nc
import pandas as pd
import numpy.ma as ma
import matplotlib as mpl
import matplotlib.pyplot as pltz
import xarray as xar
import rioxarray
from sqlalchemy import create_engine
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import sessionmaker
from sqlalchemy.orm import Mapped
from sqlalchemy import ForeignKey, Sequence, MetaData
from sqlalchemy import select, update
from sqlalchemy.orm import mapped_column, DeclarativeBase
from datetime import datetime

In [2]:
import pathlib

In [5]:
dir:str=r'./data/'
file_name:str='1epme2_2023'
pathlib.Path(dir)/file_name
full_path:str=str(pathlib.Path(dir)/file_name)
full_path

'data\\1epme2_2023'

In [6]:
with open(full_path,'rb') as f:
    data=pd.read_table(f,sep='\s+',encoding='unicode_escape',header=None,infer_datetime_format=False)
    print('读取成功')

读取成功


In [8]:
data.head()

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,-4.58,-5.48,-6.62,-7.84,-8.87,-9.31,-8.98,-8.07,-6.83,-5.48,...,-5.89,-7.16,-8.40,-9.24,-9.35,-8.78,-7.75,-6.48,-5.24,-4.41
1,-4.23,-4.69,-5.55,-6.63,-7.81,-8.79,-9.18,-8.83,-7.92,-6.70,...,-4.88,-5.98,-7.26,-8.48,-9.28,-9.35,-8.75,-7.72,-6.47,-5.26
2,-4.46,-4.32,-4.79,-5.64,-6.71,-7.85,-8.78,-9.09,-8.68,-7.75,...,-4.18,-5.02,-6.16,-7.44,-8.63,-9.34,-9.32,-8.66,-7.59,-6.34
3,-5.16,-4.43,-4.37,-4.90,-5.79,-6.87,-7.98,-8.82,-9.02,-8.51,...,-3.86,-4.31,-5.24,-6.43,-7.71,-8.82,-9.40,-9.24,-8.48,-7.35
4,-6.09,-4.97,-4.37,-4.44,-5.08,-6.02,-7.11,-8.19,-8.90,-8.94,...,-3.98,-3.91,-4.52,-5.56,-6.79,-8.03,-9.02,-9.42,-9.08,-8.19


In [17]:
data.iloc[0,1]

-5.48

In [10]:
import arrow
start_dt_ar:arrow.Arrow=arrow.get(2023,1,1,0,0)
start_dt_ar

<Arrow [2023-01-01T00:00:00+00:00]>

In [19]:
index_days=0
list_days=[]
list_realdata=[]
for day in range(data.shape[0]):
    current_day_ar:arrow.Arrow=start_dt_ar.shift(days=day)
    list_days.append(current_day_ar)
    for hour in range(data.shape[1]):
        current_day_dt_ar:arrow.Arrow=current_day_ar.shift(hours=hour)
        temp_val=data.iloc[day,hour]
        temp_={'dt':current_day_dt_ar,'val':temp_val}
        list_realdata.append(temp_)   
        
    
    

In [20]:
list_realdata[:5]

[{'dt': <Arrow [2023-01-01T00:00:00+00:00]>, 'val': -4.58},
 {'dt': <Arrow [2023-01-01T01:00:00+00:00]>, 'val': -5.48},
 {'dt': <Arrow [2023-01-01T02:00:00+00:00]>, 'val': -6.62},
 {'dt': <Arrow [2023-01-01T03:00:00+00:00]>, 'val': -7.84},
 {'dt': <Arrow [2023-01-01T04:00:00+00:00]>, 'val': -8.87}]

In [21]:
data.head()

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,-4.58,-5.48,-6.62,-7.84,-8.87,-9.31,-8.98,-8.07,-6.83,-5.48,...,-5.89,-7.16,-8.40,-9.24,-9.35,-8.78,-7.75,-6.48,-5.24,-4.41
1,-4.23,-4.69,-5.55,-6.63,-7.81,-8.79,-9.18,-8.83,-7.92,-6.70,...,-4.88,-5.98,-7.26,-8.48,-9.28,-9.35,-8.75,-7.72,-6.47,-5.26
2,-4.46,-4.32,-4.79,-5.64,-6.71,-7.85,-8.78,-9.09,-8.68,-7.75,...,-4.18,-5.02,-6.16,-7.44,-8.63,-9.34,-9.32,-8.66,-7.59,-6.34
3,-5.16,-4.43,-4.37,-4.90,-5.79,-6.87,-7.98,-8.82,-9.02,-8.51,...,-3.86,-4.31,-5.24,-6.43,-7.71,-8.82,-9.40,-9.24,-8.48,-7.35
4,-6.09,-4.97,-4.37,-4.44,-5.08,-6.02,-7.11,-8.19,-8.90,-8.94,...,-3.98,-3.91,-4.52,-5.56,-6.79,-8.03,-9.02,-9.42,-9.08,-8.19


#### 定义 `sqlalchemy` `model` 类

In [37]:
from sqlalchemy import create_engine
from sqlalchemy import Column, Date, Float, ForeignKey, Integer, text
from sqlalchemy.dialects.mysql import DATETIME, INTEGER, TINYINT, VARCHAR
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import ForeignKey, Sequence, MetaData, Table
from sqlalchemy.orm import relationship, sessionmaker
from datetime import datetime

In [24]:
DATABASES = {
    'default': {
        'ENGINE': 'mysqldb',  # 数据库引擎
        'NAME': 'surge_global_sys',  # 数据库名
        'USER': 'root',  # 账号
        'PASSWORD': '123456',
        'HOST': '127.0.0.1',  # HOST
        'POST': 3306,  # 端口
        'OPTIONS': {
            "init_command": "SET foreign_key_checks = 0;",
        },
    }
}

In [48]:
class DBConfig:
    """
    DbConfig DB配置类
    :version: 1.4
    :date: 2020-02-11
    """

    driver = 'mysql+mysqldb'
    host = '127.0.0.1'
    port = '3306'
    username = 'root'
    password = '123456'
    database = 'surge_global_sys'
    charset = 'utf8mb4'
    table_name_prefix = ''
    echo = False
    pool_size = 100
    max_overflow = 100
    pool_recycle = 60

    def get_url(self):
        config = [
            self.driver,
            '://',
            self.username,
            ':',
            self.password,
            '@',
            self.host,
            ':',
            self.port,
            '/',
            self.database,
            '?charset=',
            self.charset,
        ]

        return ''.join(config)

In [49]:
class DbFactory:
    """
        数据库工厂
    """

    def __init__(self, db_mapping: str = 'default', engine_str: str = None, host: str = None, db_name: str = None,
                 user: str = None,
                 pwd: str = None):
        db_options = DATABASES.get(db_mapping)
        self.engine_str = engine_str if engine_str else db_options.get('ENGINE')
        self.host = host if host else db_options.get('HOST')
        self.db_name = db_name if db_name else db_options.get('NAME')
        self.user = user if user else db_options.get('USER')
        self.password = pwd if pwd else db_options.get('PASSWORD')
        
        config=DBConfig()
        
        self.engine = create_engine(
            config.get_url(),
            pool_size=config.pool_size,
            max_overflow=config.max_overflow,
            pool_recycle=config.pool_recycle,
            echo=config.echo
        )
        self._session_def = sessionmaker(bind=self.engine)

    @property
    def Session(self) -> sessionmaker:
        if self._session_def is None:
            self._session_def = sessionmaker(bind=self.engine)
        return self._session_def()

In [50]:
engine = DbFactory().engine
session=DbFactory().Session
# # 生成基类
# BaseMeta = declarative_base()
# md = MetaData(bind=engine)  # 引用MetaData
# metadata = BaseMeta.metadata

In [51]:
class BaseMeta(DeclarativeBase):
    pass

In [52]:
class IIdModel(BaseMeta):
    __abstract__ = True
    id = Column(Integer, primary_key=True)


class IDel(BaseMeta):
    """
        软删除 抽象父类
    """
    __abstract__ = True
    is_del = Column(TINYINT(1), nullable=False, server_default=text("'0'"), default=0)


class IModel(BaseMeta):
    """
        model 抽象父类，主要包含 创建及修改时间
    """
    __abstract__ = True
    gmt_create_time = Column(DATETIME(fsp=6), default=datetime.utcnow())
    gmt_modify_time = Column(DATETIME(fsp=6), default=datetime.utcnow())

class StationAstronomicTideRealDataModel(IIdModel, IDel, IModel):
    """
        天文潮
    """
    __tablename__ = 'station_astronomic_tide'
    station_code = Column(VARCHAR(200), nullable=False)
    forecast_dt = Column(DATETIME(fsp=2))
    surge = Column(Float, nullable=False)

In [53]:
arrow.now().datetime

datetime.datetime(2023, 4, 6, 20, 48, 12, 310965, tzinfo=tzlocal())

In [54]:
station_code='epme2'
for temp_ in list_realdata:
    temp_model=StationAstronomicTideRealDataModel(station_code=station_code,forecast_dt=temp_['dt'].datetime,surge=temp_['val'])
    session.add(temp_model)
session.commit()
print(f'写入{station_code}成功!')
    

写入epme2成功!
